In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
data_new=pd.read_csv('C:\\Users\\Administrator\\Desktop\\data\\data_new.csv')
data_new.drop('source',axis=1,inplace=True)#source列值都是xs 可删除该特征
data_new
data_new.shape

(4754, 90)

In [4]:
y_features=[]#存放y特征
x_features=[]#存放x特征
for col in data_new.columns:
    if col=='status':
        y_features.append(col)
    else:
        x_features.append(col)

X=data_new.loc[:,x_features]
y=data_new.loc[:,y_features]


In [5]:
import math
from sklearn.utils.multiclass import type_of_target
from scipy import stats

In [9]:
# 求woe值和iv值
def woe(X, y, event):
    res_woe = []  #列表存放woe字典
    res_iv = []  #列表存放iv
    X1,c = feature_discretion(X) #对连续型特征进行处理


    for i in range(0, X1.shape[-1]):  #遍历所有特征
        x = X1[:, i]  #单个特征
        woe_dict, iv1 = woe_single_x(x, y, event)  #计算单个特征的woe值
        res_woe.append(woe_dict)
        res_iv.append(iv1)
    col = (X.columns)
    r=list(zip(col,res_iv))



    return  r #返回数组

#求单个特征的woe值
def woe_single_x(x, y, event):
    event_total=0
    for v in y.values:
        v=int(v)
        if v==event:
            event_total+=1


    non_event_total = y.shape[-1] - event_total

    iv = 0
    woe_dict = {}
    for x1 in set(x):  # 遍历各个块
        y1 = y.reindex(np.where(x == x1)[0])
        event_count = 0
        for v in y1.values:
            if v == event:
                event_count += 1
        non_event_count = y1.shape[-1] - event_count
        rate_event = event_count / event_total
        rate_non_event = non_event_count / non_event_total

        if rate_event == 0:
            rate_event = 0.0001
            # woei = -20
        elif rate_non_event == 0:
            rate_non_event = 0.0001
            # woei = 20
        woei = math.log(rate_event / rate_non_event)
        woe_dict[x1] = woei
        iv += (rate_event - rate_non_event) * woei
    return woe_dict, iv
#计算个数
def count_binary(a, event):
    event_count = (a == event).sum()
    non_event_count = a.shape[-1] - event_count
    return event_count, non_event_count

#判断特征数据是否为离散型
def feature_discretion(X):
    temp = []
    y_temp=[]
    for i in X.columns:
        x=X[i].values
        y=i


        x_type = type_of_target(x)
        if pd.Series(list(x)).dtype != 'O':
            x1 = discrete(x)
            temp.append(x1)
            y_temp.append(y)
        else:
            temp.append(x)
            y_temp.append(y)
    return np.array(temp).T,y_temp

#对连续型特征进行离散化
def discrete(x):
    res = np.array([0] * x.shape[-1], dtype=int)
    for i in range(5):
        point1 = stats.scoreatpercentile(x, i * 20)
        point2 = stats.scoreatpercentile(x, (i + 1) * 20)
        x1 = x[np.where((x >= point1) & (x <= point2))]
        mask = np.in1d(x, x1)
        res[mask] = (i + 1)
    return res



In [11]:
woe(X, y, event=1)

[('low_volume_percent', 7.738541531501908e-06),
 ('middle_volume_percent', 1.263200708246697e-05),
 ('take_amount_in_later_12_month_highest', 7.82119456953713e-06),
 ('trans_amount_increase_rate_lately', 1.1478141792260898e-05),
 ('trans_activity_month', 1.353963180898566e-05),
 ('trans_activity_day', 1.13049733195945e-05),
 ('transd_mcc', 1.307965480389311e-05),
 ('trans_days_interval_filter', 1.174246663771151e-05),
 ('trans_days_interval', 1.1764932468154797e-05),
 ('regional_mobility', 6.5362360567142086e-06),
 ('student_feature', 8.876899218002586e-07),
 ('repayment_capability', 1.1302794346983191e-05),
 ('is_high_user', 0.0),
 ('number_of_trans_from_2011', 1.1407572128608823e-05),
 ('historical_trans_amount', 1.1407543301305762e-05),
 ('historical_trans_day', 1.1405457544884586e-05),
 ('rank_trad_1_month', 1.3359410307508114e-05),
 ('trans_amount_3_month', 1.1632109310207241e-05),
 ('avg_consume_less_12_valid_month', 6.5765977211092795e-06),
 ('abs', 1.1330178802169424e-05),
 ('t

In [15]:
#使用随机森林选取特征
#平均不纯度减少 mean decrease impurity
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)#划分数据集
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
feature1 = sorted(zip(map(lambda x: '%.4f'%x, rf.feature_importances_), list(X_train.columns)), reverse=True)

len(feature1)
feature1


[('0.1063', 'trans_fail_top_count_enum_last_1_month'),
 ('0.0482', 'history_fail_fee'),
 ('0.0460', 'loans_score'),
 ('0.0328', 'trans_amount_3_month'),
 ('0.0232', 'trans_day_last_12_month'),
 ('0.0219', 'repayment_capability'),
 ('0.0202', 'trans_amount_increase_rate_lately'),
 ('0.0191', 'loans_overdue_count'),
 ('0.0181', 'apply_score'),
 ('0.0163', 'trans_days_interval'),
 ('0.0159', 'first_transaction_day'),
 ('0.0158', 'loans_long_time'),
 ('0.0156', 'number_of_trans_from_2011'),
 ('0.0155', 'latest_query_day'),
 ('0.0145', 'loans_latest_day'),
 ('0.0145', 'latest_one_month_fail'),
 ('0.0144', 'historical_trans_day'),
 ('0.0143', 'avg_price_last_12_month'),
 ('0.0140', 'latest_one_month_suc'),
 ('0.0136', 'apply_credibility'),
 ('0.0133', 'consume_mini_time_last_1_month'),
 ('0.0130', 'consume_top_time_last_6_month'),
 ('0.0130', 'abs'),
 ('0.0126', 'trans_activity_month'),
 ('0.0125', 'consume_top_time_last_1_month'),
 ('0.0123', 'loans_avg_limit'),
 ('0.0122', 'trans_fail_top_

In [16]:
#平均精确率减少 Mean decrease accuracy
import numpy as np
from collections import defaultdict
from sklearn.model_selection import cross_val_score, ShuffleSplit

from sklearn.metrics import r2_score

scores = defaultdict(list)
rs = ShuffleSplit(n_splits=5, test_size=0.3, random_state=2018)
for train_idx, test_idx in rs.split(X_train):
    x_train, x_test = X_train.values[train_idx], X_train.values[test_idx]
    Y_train, Y_test = y_train.values[train_idx], y_train.values[test_idx]
    r = rf.fit(x_train, Y_train)
    acc = r2_score(Y_test, rf.predict(x_test))
    for i in range(x_train.shape[1]):
        X_t = x_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[X_train.columns[i]].append((acc - shuff_acc) / acc)
        
feature2=sorted([('%.4f'%abs(np.mean(score)), feat) for feat, score in scores.items()], reverse=True)

len(feature2)
feature2

[('1.2901', 'trans_fail_top_count_enum_last_1_month'),
 ('0.7537', 'history_fail_fee'),
 ('0.1759', 'loans_overdue_count'),
 ('0.1065', 'latest_one_month_fail'),
 ('0.1053', 'loans_score'),
 ('0.1013', 'trans_day_last_12_month'),
 ('0.1005', 'trans_amount_3_month'),
 ('0.0909', 'history_suc_fee'),
 ('0.0792', 'historical_trans_amount'),
 ('0.0698', 'apply_score'),
 ('0.0541', 'transd_mcc'),
 ('0.0523', 'max_consume_count_later_6_month'),
 ('0.0478', 'latest_one_month_suc'),
 ('0.0457', 'loans_long_time'),
 ('0.0416', 'latest_query_day'),
 ('0.0414', 'max_cumulative_consume_later_1_month'),
 ('0.0341', 'consume_top_time_last_6_month'),
 ('0.0331', 'repayment_capability'),
 ('0.0291', 'low_volume_percent'),
 ('0.0255', 'pawns_auctions_trusts_consume_last_1_month'),
 ('0.0247', 'pawns_auctions_trusts_consume_last_6_month'),
 ('0.0241', 'historical_trans_day'),
 ('0.0232', 'consfin_credibility'),
 ('0.0230', 'latest_three_month_loan'),
 ('0.0230', 'avg_price_last_12_month'),
 ('0.0204', 'f

In [18]:
#根据之前两个算的相关性的值，综合选择特征，选择两种排列后四十个的数据 作比较
drop_feature = []#要删除的特征
for feature in X_train.columns:
    if feature in [t[1] for t in feature1[50:]] and feature in [t[1] for t in feature2[50:]]:
        drop_feature.append(feature)

drop_feature

['regional_mobility',
 'student_feature',
 'is_high_user',
 'top_trans_count_last_1_month',
 'reg_preference_for_trad',
 'trans_fail_top_count_enum_last_12_month',
 'railway_consume_count_last_12_month',
 'jewelry_consume_count_last_6_month',
 'query_org_count',
 'query_cash_count',
 'latest_one_month_apply',
 'latest_six_month_apply',
 'loans_count',
 'loans_org_count_behavior',
 'consfin_org_count_behavior',
 'loans_cash_count',
 'latest_one_month_loan',
 'loans_credit_limit',
 'loans_org_count_current',
 'loans_product_count',
 'consfin_credit_limit',
 'first_transaction_time_year',
 'latest_query_time_year',
 'latest_query_time_month',
 'latest_query_time_weekday',
 'loans_latest_time_year']

In [19]:
X_train.drop(drop_feature, axis = 1, inplace = True)
X_test.drop(drop_feature, axis = 1, inplace = True)

In [21]:
X_train.columns#剩下的特征


Index(['low_volume_percent', 'middle_volume_percent',
       'take_amount_in_later_12_month_highest',
       'trans_amount_increase_rate_lately', 'trans_activity_month',
       'trans_activity_day', 'transd_mcc', 'trans_days_interval_filter',
       'trans_days_interval', 'repayment_capability',
       'number_of_trans_from_2011', 'historical_trans_amount',
       'historical_trans_day', 'rank_trad_1_month', 'trans_amount_3_month',
       'avg_consume_less_12_valid_month', 'abs', 'avg_price_last_12_month',
       'avg_price_top_last_12_valid_month', 'trans_top_time_last_1_month',
       'trans_top_time_last_6_month', 'consume_top_time_last_1_month',
       'consume_top_time_last_6_month', 'cross_consume_count_last_1_month',
       'trans_fail_top_count_enum_last_1_month',
       'trans_fail_top_count_enum_last_6_month',
       'consume_mini_time_last_1_month',
       'max_cumulative_consume_later_1_month',
       'max_consume_count_later_6_month',
       'pawns_auctions_trusts_consume_

In [22]:
len(X_train.columns)#剩下的特征个数


63